## 4. LangChain解説

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)

result = llm.invoke("自己紹介してください。")
print(result)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンと言います！"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content= "私の名前が分かりますか？")
]

result = chat.invoke(messages)
print(result.content)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

messages = [HumanMessage(content="自己紹介してください。")]

for chunk in chat.stream(messages):
    print(chunk.content)

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """以下のレシピを考えてください。

料理名: {dish}
"""
)

result = prompt.format(dish="カレー")
print(result)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは{country}料理のプロフェッショナルです。"),
    ("human", "以下の料理のレシピを考えてください。\n\n料理名: {dish}")
])

messages = chat_prompt.format_messages(country="イギリス", dish="肉じゃが")

print(messages)

In [ ]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)
result = chat.invoke(messages)
print(result.content)

In [12]:
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

In [30]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "以下の料理のレシピを考えてください。\n\n料理名: {dish}")
    ]
)

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0).with_structured_output(Recipe)

chain = prompt | chat

output = chain.invoke({"dish": "カレー"})

In [ ]:
print(output)

In [ ]:
output.model_dump()

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

cot_prompt = PromptTemplate.from_template(
    """以下の質問に回答してください。

質問: {question}

ステップバイステップで考えましょう。
"""
)

cot_chain = (
    {"question": RunnablePassthrough()}
    | cot_prompt
    | model
    | StrOutputParser()
)

summarize_prompt = PromptTemplate.from_template(
    """以下の文章の結論だけを一言に要約してください。

{input}"""
)

summarize_chain = (
    {"input": RunnablePassthrough()}
    | summarize_prompt
    | model
    | StrOutputParser()
)

cot_summarize_chain = cot_chain | summarize_chain

result = cot_summarize_chain.invoke(
    "私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？"
)
print(result)

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.memory import ConversationBufferMemory

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ]
)

memory = ConversationBufferMemory(return_messages=True)

chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
    | StrOutputParser()
)

while True:
    user_message = input("You: ")
    inputs = {"input": user_message}

    ai_message = chain.invoke(inputs)
    memory.save_context(inputs, {"output": ai_message})

    print(ai_message)

## LangChainの活用

In [ ]:
# Document loaders
from langchain.document_loaders import GitLoader


def file_filter(file_path):
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))

In [ ]:
# Document transformers
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(raw_docs)
print(len(docs))

In [5]:
# Text embedding models
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
query = "AWSのS3からデータを読み込むためのDocumentLoaderはありますか？"

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

In [ ]:
# Vector stores
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(docs, embeddings)
db

In [8]:
# Retriever
retriever = db.as_retriever()

In [ ]:
query = "AWSのS3からデータを読み込むためのDocumentLoaderはありますか？"

context_docs = retriever.invoke(query)
print(f"len = {len(context_docs)}")

first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)

In [ ]:
# LCEL RAG
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    """以下の文脈だけをふまえて質問に回答してください。

{context}

Question: {question}
""")

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

result = chain.invoke(query)
print(result)

In [15]:
from langchain_core.globals import set_verbose

set_verbose(True)

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)
tools = load_tools(["terminal"], allow_dangerous_tools=True)
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(chat, tools, prompt)

agent_chain = AgentExecutor(agent=agent, tools=tools)

result = agent_chain.invoke({"input": "このディレクトリにあるファイルの一覧を教えて"})
print(result["output"])

In [20]:
from langchain_core.tools import Tool

def my_super_func(param):
    return "42"

tools = [
    Tool.from_function(
        func=my_super_func,
        name="The_Answer",
        description="生命、宇宙、そして万物についての究極の疑問の答え"
    )
]

In [ ]:
agent = create_react_agent(chat, tools, prompt)
agent_chain = AgentExecutor(agent=agent, tools=tools)

result = agent_chain.invoke({"input": "この世界の真理を教えてください"})
print(result["output"])

In [23]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

summarize_prompt = PromptTemplate.from_template(
    """以下の文章を結論だけ一言に要約してください。

{input}"""
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

summarize_chain = (
    {"input": RunnablePassthrough()}
    | summarize_prompt
    | model
    | StrOutputParser()
)

tools = [
    Tool.from_function(
        func=summarize_chain.invoke,
        name="Summarizer",
        description="Text summarizer"
    )
]

In [ ]:
summarize_chain.invoke({"input": "これはテストの文章です"})

In [ ]:
agent = create_react_agent(chat, tools, prompt)
agent_chain = AgentExecutor(agent=agent, tools=tools)

text = """以下を要約してください。

こんにちは！私はChatGPTと呼ばれるAI言語モデルです。OpenAIが開発したGPT-3.5アーキテクチャに基づいています。私は自然言語理解と生成に特化しており、さまざまなトピックに関する質問に答えたり、おしゃべりしたりすることが得意です。
私のトレーニングデータは2021年9月までの情報に基づいているため、それ以降の出来事については知識がありません。ですが、できる限りお手伝いすることに努めます。
質問や会話、情報の共有など、どんなお手伝いでもお気軽にお申し付けください！よろしくお願いします。"""

result = agent_chain.invoke({"input": text})
print(result["output"])

In [ ]:
# Function callingを使うOpenAI Functions Agent
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent, load_tools
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
tools = load_tools(["terminal"], allow_dangerous_tools=True)
prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(chat, tools, prompt)
agent_chain = AgentExecutor(agent=agent, tools=tools)

result = agent_chain.invoke({"input": "このディレクトリにあるファイルの一覧を教えて"})
print(result["output"])

In [ ]:
# 複数のツールを一度に使うOpenAI Multi Functions Agent
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent, load_tools
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
tools = load_tools(["ddg-search"])
prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(chat, tools, prompt)
agent_chain = AgentExecutor(agent=agent, tools=tools)

result = agent_chain.invoke({"input": "東京と大阪の天気を教えて"})
print(result["output"])

In [ ]:
# Function callingを応用したOutputParser
import json
from typing import Optional

from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel

class Person(BaseModel):
    person_name: str
    person_height: str
    person_hair_color: Optional[str]
    dog_name: Optional[str]
    dog_breed: Optional[str]

class People(BaseModel):
    people: list[Person]


model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0).bind_tools([People])

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant."),
    ("user", "{input}")
])

chain = prompt | model | JsonOutputToolsParser()

text = """
Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.
Alex's dog Frosty is a labrador and likes to play hide and seek.
"""

people = chain.invoke({"input": text})
print(people)

In [ ]:
print(json.dumps(people, indent=2))

In [ ]:
# Evaluation
from langchain.evaluation import load_evaluator
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

evaluator = load_evaluator("qa", eval_llm=chat)

result = evaluator.evaluate_strings(
    input="私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？",
    prediction="""1最初に10個のリンゴを買い、その中から隣人と修理工にそれぞれ2個ずつ渡しました。そのため、まず手元に残ったリンゴは10 - 2 - 2 = 6個となります。

その後、さらに5個のリンゴを買い、1つ食べました。これにより手元のリンゴは6 + 5 - 1 = 10個となります。""",
    reference="10個",
)

print(result)